In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.initializers import RandomUniform
from keras.initializers import glorot_uniform

import tensorflow as tf
from sklearn.metrics import roc_auc_score

def roc_auc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

Using TensorFlow backend.


In [2]:
column_names = ['id', 'tdate', 'tname', 'ttext', 'ttype', 'trep', 'trtw', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount']
df_positive = pd.read_csv('data/tweets/positive.csv', delimiter=';', names=column_names, index_col='id')
df_negative = pd.read_csv('data/tweets/negative.csv', delimiter=';', names=column_names, index_col='id')

In [3]:
df_positive

,tdate,tname,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
id,,,,,,,,,,,
408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...
411368729235054592,1386912922,diminlisenok,"Спала в родительском доме, на своей кровати......",1,0,0,0,1497,56,34,2
411368729424187392,1386912922,qilepocagotu,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,0,1,0,692,225,210,0
411368796537257984,1386912938,DennyChooo,"Что происходит со мной, когда в эфире #proacti...",1,0,0,0,4905,448,193,13


In [4]:
df_negative

,tdate,tname,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
id,,,,,,,,,,,
408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0
...,...,...,...,...,...,...,...,...,...,...,...
425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0


In [5]:
df_positive['ttype'].unique()

array([1])

In [6]:
df_negative['ttype'].unique()

array([-1])

In [7]:
usecols = ['ttext', 'ttype']
df = pd.concat((df_positive[usecols], df_negative[usecols])).sample(frac=1, random_state=42).reset_index(drop=True)

In [8]:
df

,ttext,ttype
0,мыс на меня обиделась:(\nя ей даже ничего не с...,-1
1,"аааааааааааааааааааа,не хочу на работу :(",-1
2,"У меня какой-то особенный вид ушей! :D, некото...",1
3,@simonovkon он неплохой человек в жизни. Я ра...,-1
4,"RT @Darina_Lo: Домааааа\nЕхали на такси, пели ...",1
...,...,...
226829,"Хочу, чтобы графический планшетик скорее прише...",-1
226830,RT @OmenDougnter_: мы с @fluffy_irk @hamsterka...,1
226831,дому чуть больше 30 лет а мне приходится втору...,-1
226832,@MIKEFUCKINGWAY @_im_killjoy_ даже представить...,-1


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df['ttext'], df['ttype'], test_size=0.25, shuffle=True, random_state=42, stratify=df['ttype'])

In [10]:
X_train.str.split(' ').apply(len).describe(percentiles = [0.25, 0.50, 0.75, 0.90])

count    170125.000000
mean         12.175812
std           4.871840
min           1.000000
25%           8.000000
50%          11.000000
75%          15.000000
90%          19.000000
max          90.000000
Name: ttext, dtype: float64

_Без предобработки_ 90% твитов содержат 19 и меньше слов.

In [11]:
max_twords = X_train.str.split(' ').apply(len).max()

In [12]:
word_counts = Counter(np.concatenate(X_train.str.split(' ').values))

In [13]:
sum(word_counts.values())

2071410

In [14]:
len(word_counts)

472075

In [15]:
sum(word_counts.values()) / len(word_counts)

4.387883281258275

In [16]:
word_counts.most_common()

[('не', 50415),
 ('и', 39433),
 ('в', 39019),
 ('я', 35436),
 ('RT', 27923),
 ('на', 26362),
 ('что', 19748),
 ('с', 18712),
 ('а', 15770),
 ('', 14265),
 ('меня', 13775),
 (':(', 13417),
 ('у', 13395),
 ('как', 11809),
 ('это', 11058),
 ('мне', 10931),
 ('так', 10620),
 ('все', 9704),
 ('ты', 8683),
 ('по', 8334),
 (':)', 8332),
 ('Я', 7390),
 ('за', 7299),
 ('-', 7197),
 ('но', 7186),
 ('уже', 6981),
 ('то', 6844),
 (':D', 6566),
 ('же', 6482),
 ('бы', 5714),
 ('только', 5579),
 ('еще', 5442),
 ('ну', 5255),
 ('А', 5232),
 ('к', 4660),
 ('вот', 4486),
 ('когда', 4331),
 ('тебя', 4277),
 ('очень', 4197),
 ('от', 4140),
 ('он', 4082),
 ('сегодня', 4074),
 ('из', 3992),
 ('просто', 3991),
 ('(', 3775),
 ('мы', 3717),
 ('будет', 3701),
 ('—', 3608),
 ('хочу', 3563),
 (',', 3491),
 ('до', 3412),
 ('тебе', 3303),
 ('тоже', 3286),
 ('И', 3242),
 ('даже', 3224),
 ('да', 3186),
 ('было', 3168),
 ('день', 3092),
 ('для', 3088),
 ('его', 3083),
 ('там', 3013),
 ('теперь', 2855),
 ('если', 2798)

Видно, что чаще всего встречаются стоп-слова. Не будем их обрабатывать сейчас.

In [17]:
pd.Series(word_counts).describe(percentiles=[0.75, 0.9, 0.95, 0.99])

count    472075.000000
mean          4.387883
std         161.054539
min           1.000000
50%           1.000000
75%           1.000000
90%           3.000000
95%           6.000000
99%          29.000000
max       50415.000000
dtype: float64

По крайней мере 75% слов встречаются только 1 раз. 90% - 3 и меньше раза. Рассмотрим топ-10% слов.

In [18]:
max_words = int(len(word_counts) * 0.1)

In [19]:
def transform(X, word_counts, max_words, max_len):
    X = X.str.split(' ')
    ids = dict((word, min(i, max_words) + 1) for (i, (word, _)) in enumerate(word_counts.most_common()))
    X = X.apply(lambda x: [ids[y] if y in ids else max_words for y in x])
    return pad_sequences(X, truncating='pre', padding='pre', maxlen=max_len)

In [20]:
X_train_transformed = transform(X_train, word_counts, max_words, max_twords)

In [29]:
embedding_dim = 32
model = Sequential([
    Embedding(max_words + 2, embedding_dim, mask_zero=True, embeddings_initializer=RandomUniform(-0.01, 0.01, seed=42)),
    LSTM(128, kernel_initializer=glorot_uniform(seed=42), dropout=0.1, recurrent_dropout=0.1),
    Dense(1, activation='tanh')
])

In [30]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[roc_auc])

In [31]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 32)          1510688   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 1,593,249
Trainable params: 1,593,249
Non-trainable params: 0
_________________________________________________________________


In [32]:
batch_size = 2000
max_epochs = 10
model.fit(X_train_transformed, y_train, epochs=max_epochs, batch_size=batch_size, shuffle=True, verbose=1)

/home/nikolai/Projects/MIPT-ML-Fall-2019/env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
170125/170125 [==============================] - 204s 1ms/step - loss: -1.6776 - roc_auc: 0.8060
Epoch 2/10
170125/170125 [==============================] - 202s 1ms/step - loss: -3.5141 - roc_auc: 0.9051
Epoch 3/10
170125/170125 [==============================] - 201s 1ms/step - loss: -4.5223 - roc_auc: 0.9242
Epoch 4/10
170125/170125 [==============================] - 201s 1ms/step - loss: -4.5290 - roc_auc: 0.9277
Epoch 5/10
170125/170125 [==============================] - 201s 1ms/step - loss: -4.8458 - roc_auc: 0.9292
Epoch 6/10
170125/170125 [==============================] - 201s 1ms/step - loss: -5.0034 - roc_auc: 0.9375
Epoch 7/10
170125/170125 [==============================] - 201s 1ms/step - loss: -4.7172 - roc_auc: 0.9447
Epoch 8/10
170125/170125 [==============================] - 200s 1ms/step - loss: -5.2685 - roc_auc: 0.9467
Epoch 9/10
170125/170125 [==============================] - 201s 1ms/step - loss: -4.6488 - roc_auc: 0.9370
Epoch 10/10
170125/170125 [=

In [33]:
X_test_transformed = transform(X_test, word_counts, max_words, max_twords)

In [34]:
model.evaluate(X_test_transformed, y_test)

56709/56709 [==============================] - 39s 691us/step


[-3.6233027854780584, 0.8405951261520386]

Предобработка текста:

In [35]:
from pymystem3 import Mystem
stemmer = Mystem()

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian') + stopwords.words('english'))

import re
space_characters = re.compile(r"[\s_]+")
email = re.compile(r"\w+([-+.']\w+)*@\w+([-.]\w+)*\.\w+([-.]\w+)*")
url = re.compile(r"http[s]?:\/\/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")
username = re.compile(r"@[\w_]+")
allowed_tokens = re.compile(r"[\w']+|[!?:'(\)@\+\-]")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nikolai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
def process(string):
    string = string.lower()
    string = re.sub(email, 'email', string)
    string = re.sub(url, 'url', string)
    string = re.sub(username, 'username', string)
    string = re.sub(space_characters, ' ', string)
    # Оставим пунктуацию: мы пользуемся нейронной моделью
    # + смайлы вроде :) и :( имеют смысл для текущей задачи
    string = ' '.join(re.findall(allowed_tokens, string))
    string = ''.join(stemmer.lemmatize(string))
    string = string.strip()
    return string

In [37]:
%%time
X_train_processed = X_train.apply(process)
X_train_processed = X_train_processed.str.split(' ').apply(lambda x: ' '.join(y for y in x if y not in stop_words))

CPU times: user 24.6 s, sys: 2.22 s, total: 26.8 s
Wall time: 1min 39s


In [38]:
X_train_processed.str.split(' ').apply(len).describe(percentiles = [0.25, 0.50, 0.75, 0.90])

count    170125.000000
mean         10.727877
std           4.522635
min           1.000000
25%           8.000000
50%          10.000000
75%          13.000000
90%          16.000000
max         130.000000
Name: ttext, dtype: float64

In [39]:
max_twords = X_train.str.split(' ').apply(len).max()

In [40]:
word_counts = Counter(np.concatenate(X_train_processed.str.split(' ').values))

In [41]:
sum(word_counts.values())

1825080

In [42]:
len(word_counts)

89395

In [43]:
sum(word_counts.values()) / len(word_counts)

20.41590692991778

In [44]:
word_counts.most_common()

[('(', 159350),
 (')', 146082),
 ('username', 111853),
 (':', 111180),
 ('!', 50201),
 ('-', 38541),
 ('rt', 28645),
 ('?', 26869),
 ('url', 25001),
 ('это', 16128),
 ('весь', 9088),
 ('хотеть', 8872),
 ('день', 8389),
 ('сегодня', 6633),
 ('мочь', 6391),
 ('очень', 5728),
 ('знать', 5257),
 ('просто', 5227),
 ('год', 5042),
 ('любить', 4568),
 ('человек', 4512),
 ('свой', 4221),
 ('3', 4204),
 ('завтра', 4017),
 ('новый', 3911),
 ('хороший', 3804),
 ('вообще', 3638),
 ('самый', 3443),
 ('делать', 3436),
 ('спасибо', 3291),
 ('понимать', 3261),
 ('спать', 3115),
 ('блин', 3095),
 ('сказать', 3087),
 ('почему', 3074),
 ('утро', 3042),
 ('думать', 3028),
 ('который', 3021),
 ('смотреть', 3013),
 ('время', 2999),
 ('идти', 2840),
 ('2', 2802),
 ('говорить', 2793),
 ('пойти', 2724),
 ('сидеть', 2599),
 ('давать', 2592),
 ('жизнь', 2516),
 ('писать', 2513),
 ('друг', 2500),
 ('сделать', 2476),
 ('ничто', 2372),
 ('школа', 2365),
 ('настроение', 2270),
 ('скоро', 2231),
 ('ночь', 2181),
 ('м

In [45]:
pd.Series(word_counts).describe(percentiles=[0.75, 0.9, 0.95, 0.99])

count     89395.000000
mean         20.415907
std         941.129933
min           1.000000
50%           1.000000
75%           3.000000
90%          11.000000
95%          29.000000
99%         200.000000
max      159350.000000
dtype: float64

Видно, что уникальных токенов стало меньше, чаще всего встречается пунктуация, ссылки и необработанные стоп-слова. Возьмём топ-25% слов

In [46]:
max_words = int(len(word_counts) * 0.25)

In [47]:
X_train_transformed = transform(X_train_processed, word_counts, max_words, max_twords)

In [48]:
embedding_dim = 32
model = Sequential([
    Embedding(max_words + 2, embedding_dim, mask_zero=True, embeddings_initializer=RandomUniform(-0.01, 0.01, seed=42)),
    LSTM(128, kernel_initializer=glorot_uniform(seed=42), dropout=0.1, recurrent_dropout=0.1),
    Dense(1, activation='tanh')
])

In [49]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[roc_auc])

In [50]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          715200    
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 797,761
Trainable params: 797,761
Non-trainable params: 0
_________________________________________________________________


In [51]:
batch_size = 2000
max_epochs = 10
model.fit(X_train_transformed, y_train, epochs=max_epochs, batch_size=batch_size, shuffle=True, verbose=1)

/home/nikolai/Projects/MIPT-ML-Fall-2019/env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
170125/170125 [==============================] - 205s 1ms/step - loss: -4.3384 - roc_auc: 0.9826
Epoch 2/10
170125/170125 [==============================] - 200s 1ms/step - loss: -6.2912 - roc_auc: 0.9921
Epoch 3/10
170125/170125 [==============================] - 200s 1ms/step - loss: -6.3722 - roc_auc: 0.9929
Epoch 4/10
170125/170125 [==============================] - 200s 1ms/step - loss: -6.9156 - roc_auc: 0.9970
Epoch 5/10
170125/170125 [==============================] - 199s 1ms/step - loss: -7.1132 - roc_auc: 0.9976
Epoch 6/10
170125/170125 [==============================] - 199s 1ms/step - loss: -7.2124 - roc_auc: 0.9981
Epoch 7/10
170125/170125 [==============================] - 199s 1ms/step - loss: -7.2672 - roc_auc: 0.9987
Epoch 8/10
170125/170125 [==============================] - 200s 1ms/step - loss: -7.2772 - roc_auc: 0.9988
Epoch 9/10
170125/170125 [==============================] - 200s 1ms/step - loss: -7.2004 - roc_auc: 0.9983
Epoch 10/10
170125/170125 [=

In [52]:
%%time
X_test_processed = X_test.apply(process)
X_test_processed = X_test_processed.str.split(' ').apply(lambda x: ' '.join(y for y in x if y not in stop_words))
X_test_transformed = transform(X_test_processed, word_counts, max_words, max_twords)

CPU times: user 8.84 s, sys: 911 ms, total: 9.75 s
Wall time: 39.5 s


In [53]:
model.evaluate(X_test_transformed, y_test)

56709/56709 [==============================] - 41s 727us/step


[-7.010739052562024, 0.9941747784614563]